In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
# brand model switch
brand_model_map = pd.read_csv('car models and brands (3).csv')

brand_model_map['Brand'] = brand_model_map['Brand'].astype('string')
brand_model_map['Model'] = brand_model_map['Model'].astype('string')

brand_model_map['Model'].apply(str.lower)
brand_model_map['Brand'].apply(str.lower)

brand_model_map

,Brand,Model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx
...,...,...
525,volvo,xc90
526,volvo,s60
527,volvo,s80
528,volvo,v60


In [3]:
data = pd.read_csv('edmunds_forum.csv')
data['comments'] = data['comments'].str[1:]
data.head()

,Unnamed: 0,Date,comments
0,0,"March 24, 2002 10:54PM",I personally think that with a few tweaks the ...
1,1,"March 25, 2002 12:06AM",I am debating a new purchase and these two are...
2,2,"March 25, 2002 10:02AM","Great handling, RWD, excellent engine and the ..."
3,3,"March 25, 2002 4:04PM",And no manual tranny. That may not matter to y...
4,4,"March 25, 2002 5:44PM",One beauty of BMW 3 Series is that there are s...


In [4]:
def punctuation_extermination(s):
    o = str.maketrans('', '', string.punctuation)
    return str(s).translate(o)

data['comments'] = data['comments'].apply(punctuation_extermination)
data.head()

,Unnamed: 0,Date,comments
0,0,"March 24, 2002 10:54PM",I personally think that with a few tweaks the ...
1,1,"March 25, 2002 12:06AM",I am debating a new purchase and these two are...
2,2,"March 25, 2002 10:02AM",Great handling RWD excellent engine and the be...
3,3,"March 25, 2002 4:04PM",And no manual tranny That may not matter to yo...
4,4,"March 25, 2002 5:44PM",One beauty of BMW 3 Series is that there are s...


In [5]:
corpus = ""
for l in data['comments']:
    l = str(l).lower()
    corpus += l


corpus = corpus.split()
corpus_without_stop = [word for word in corpus if word not in stopwords.words("english")]
corpus_without_stop[:500]

['personally',
 'think',
 'tweaks',
 'c320',
 'could',
 'also',
 'sit',
 'top',
 'group',
 'still',
 'entrylevel',
 'luxury',
 'sedan',
 'mercedes',
 'would',
 'lead',
 'believe',
 'c320',
 'sport',
 'needs',
 'suspension',
 'retuned',
 'accept',
 '17',
 'inch',
 'tires',
 'without',
 'harsh',
 'ride',
 'mb',
 'also',
 'needs',
 'stop',
 'forcing',
 'metal',
 'trim',
 'black',
 'leather',
 'everyone',
 'wants',
 'sport',
 'package',
 'c240s',
 '6speed',
 'wouldnt',
 'hurt',
 'either',
 'offered',
 'c320',
 'though',
 'needs',
 'fixed',
 'first',
 'said',
 'favorite',
 'group',
 'gasp',
 'new',
 'a4',
 '30',
 'a4',
 'easily',
 'best',
 'interior',
 'sub40k',
 'car',
 'combine',
 'stunning',
 'looks',
 'body',
 'rigid',
 'ever',
 'made',
 'personal',
 'favorite',
 'single',
 'thing',
 'dont',
 'like',
 'a4',
 'grey',
 'trim',
 'around',
 'bottom',
 'car',
 'trim',
 'doesnt',
 'match',
 'body',
 'colors',
 'body',
 'color',
 'anyway',
 'excellent',
 'car',
 'opinion',
 'best',
 '40k',
 'u

In [6]:
set(brand_model_map["Model"])
modellookup = brand_model_map.set_index("Model").to_dict()["Brand"]
modellookup

{'integra': 'acura',
 'legend': 'honda',
 'vigor': 'acura',
 'rlx': 'acura',
 'ilx': 'acura',
 'mdx': 'acura',
 'rdx': 'acura',
 'tsx': 'acura',
 'rsx': 'acura',
 'quattro': 'audi',
 'a4': 'audi',
 'a6': 'audi',
 'a5': 'audi',
 'a7': 'audi',
 'q3': 'audi',
 'q5': 'audi',
 'q7': 'audi',
 'a8': 'audi',
 'a3': 'audi',
 '3-series': 'bmw',
 '5-series': 'bmw',
 '6-series': 'bmw',
 '4-series': 'bmw',
 'xdrive': 'bmw',
 'alpina': 'bmw',
 '330ci': 'bmw',
 '328i': 'bmw',
 '330c': 'bmw',
 '335i': 'bmw',
 '335d': 'bmw',
 '528i': 'bmw',
 '535i': 'bmw',
 '128i': 'bmw',
 '640i': 'bmw',
 '3er': 'bmw',
 '5er': 'bmw',
 'm5': 'bmw',
 'm3': 'bmw',
 'm6': 'bmw',
 'bmws': 'bmw',
 'bmwesque': 'bmw',
 'park avenue': 'buick',
 'la crosse': 'buick',
 'lacrosse': 'buick',
 'century': 'toyota',
 'enclave': 'buick',
 'encore': 'buick',
 'regal': 'buick',
 'buicks': 'buick',
 'escalade': 'cadillac',
 'srx': 'cadillac',
 'xts': 'cadillac',
 'cars': 'car',
 'grand blazer': 'chevrolet',
 'trailblazer': 'chevrolet',
 '

In [8]:
def mbswitch(x):
    t = x.split()
    for word in t:
        if word in modellookup:
            x = x.replace(word, modellookup[word])
    return x

corpus_without_stop = [x.lower() for x in corpus_without_stop]
corpus_swapped = [mbswitch(x) for x in corpus_without_stop]
data['comments'] = data['comments'].apply(lambda i: i.lower())
data['comments'] = data['comments'].apply(lambda i: (mbswitch(i)))

In [12]:
brandlist = [brand_model_map["Brand"]]

# dict((i, np.array(corpus_without_stop).count(i)) for i in brandlist)

# data["comments"].groupby(brandlist.count())

freq_dict = {}
for word in corpus_swapped:
    if word not in freq_dict.keys():
        freq_dict[word] = 1
    elif word in freq_dict.keys():
        freq_dict[word] += 1
    else:
        print('error')

6527

In [40]:
brand_strings = (brand_model_map['Brand'].values)

brand_strings = set(brand_strings)
car_dict = {}
for package in brand_strings:
    if package in freq_dict.keys():
        car_dict[package] = freq_dict[package]

car_dict


{'volkswagen': 450,
 'infiniti': 1046,
 'hyundai': 357,
 'bmw': 6527,
 'acura': 1542,
 'pontiac': 177,
 'car': 11612,
 'volkwagen': 118,
 'honda': 1271,
 'seat': 1093,
 'sedan': 1935,
 'audi': 2045,
 'mitsubishi': 52,
 'subaru': 476,
 'volvo': 649,
 'lincoln': 429,
 'kia.': 1,
 'mercury': 19,
 'ford': 432,
 'kia': 39,
 'mercedes-benz': 384,
 'problem': 791,
 'mazda': 267,
 'saturn': 43,
 'dodge': 170,
 'chrysler': 153,
 'cadillac': 420,
 'suzuki': 30,
 'buick': 194,
 'toyota': 961,
 'nissan': 797,
 'mercedes': 87,
 'hyndai kia': 2,
 'chevrolet': 279}